In [1]:
# 1. 라이브러리 임포트 및 API 키 설정
# ===================================================================
import os
import urllib.request
import datetime
import json
import time
import ssl # HTTPS 인증서 문제를 우회하기 위해 추가
import socket # 타임아웃 설정을 위해 추가

# 🚨 중요: 본인의 Naver API ID와 Secret Key를 입력하세요.
client_id = 'Kc65Ibtu71R5TLRUF7lL'
client_secret = 'CdT4sLf9Ht' # ⬅️ 여기에 본인의 Secret Key를 입력하세요.

# HTTPS 인증서 검증 비활성화 (전역 설정)
ssl._create_default_https_context = ssl._create_unverified_context

# ===================================================================
# 2. 기본 함수 정의 (filter 파라미터 추가)
# API URL에 요청을 보내고 응답을 받아오는 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    # --- 추가된 부분: 웹 브라우저인 척 하기 위한 헤더 ---
    req.add_header("User-Agent", "Mozilla/5.0")

    try:
        # --- 핵심 변경 부분: timeout=10 설정 ---
        # 10초 안에 응답이 없으면 예외를 발생시키고 다음으로 넘어감
        response = urllib.request.urlopen(req, timeout=10)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(f"[{datetime.datetime.now()}] API 요청 실패: {e} | URL: {url}")
        return None

# --- 핵심 변경 부분: filter 인자 추가 ---
def getNaverImageSearch(srcText, start, display, img_filter='large'):
    base = "https://openapi.naver.com/v1/search/image"
    parameters = f"?query={urllib.parse.quote(srcText)}&start={start}&display={display}&filter={img_filter}"
    url = base + parameters
    return getRequestUrl(url)


# 네이버 이미지 검색 API를 호출하는 함수
# --- 핵심 변경 부분: filter 인자 추가 ---
def getNaverImageSearch(srcText, start, display, img_filter='large'):
    base = "https://openapi.naver.com/v1/search/image"
    parameters = f"?query={urllib.parse.quote(srcText)}&start={start}&display={display}&filter={img_filter}"
    url = base + parameters
    return getRequestUrl(url)

# ===================================================================
# 3. 이미지 크롤링 실행 (메인 로직)
# ===================================================================
# 검색할 키워드 목록
KEYWORDS = ["뿌팟퐁커리", "팟타이", "분짜"]
# 각 키워드당 다운로드할 이미지 개수
NUM_IMAGES = 400
# 이미지를 저장할 최상위 폴더 이름
SAVE_DIR = "4_naver_crawled_images1"

print("🚀 Naver 이미지 크롤링을 시작합니다...")

for keyword in KEYWORDS:
    print(f"\n[INFO] '{keyword}' 이미지 다운로드를 시작합니다...")
    download_path = os.path.join(SAVE_DIR, keyword)
    os.makedirs(download_path, exist_ok=True)

    success_count = 0
    start_index = 1
    display_size = 100

    while success_count < NUM_IMAGES and start_index < 1000:
        print(f"   -> API 요청: 시작 위치 {start_index}")

        json_string = getNaverImageSearch(keyword, start_index, display_size, img_filter='large')
        if json_string is None:
            break

        response_dict = json.loads(json_string)

        if 'items' not in response_dict or not response_dict['items']:
            print("   -> 더 이상 검색 결과가 없습니다. 다운로드를 중단합니다.")
            break

        for item in response_dict['items']:
            if success_count >= NUM_IMAGES:
                break

            try:
                image_url = item['link']
                clean_url = image_url.split('?')[0]
                file_ext = os.path.splitext(clean_url)[1] or '.jpg'
                filename = f"{keyword}_{success_count + 1}{file_ext}"
                save_path = os.path.join(download_path, filename)

                req = urllib.request.Request(image_url, headers={"User-Agent": "Mozilla/5.0"})
                # --- 핵심 변경 부분: timeout=10 설정 ---
                with urllib.request.urlopen(req, timeout=10) as response, open(save_path, 'wb') as out_file:
                    out_file.write(response.read())

                success_count += 1

            except socket.timeout:
                print(f"   -> 다운로드 실패 (Timeout): {image_url}")
            except Exception as e:
                print(f"   -> 다운로드 실패: {image_url} | 오류: {type(e).__name__}")

        start_index += display_size
        time.sleep(0.2)

    print(f"✅ '{keyword}' 이미지 {success_count}장 다운로드 완료! -> 저장 경로: {download_path}")

print("\n🎉 모든 Naver 이미지 크롤링 작업이 완료되었습니다.")

🚀 Naver 이미지 크롤링을 시작합니다...

[INFO] '뿌팟퐁커리' 이미지 다운로드를 시작합니다...
   -> API 요청: 시작 위치 1
   -> 다운로드 실패: https://homecuisine.co.kr/files/attach/images/142/034/048/d65437be8115189cf060cf8c01e06ec6.JPG | 오류: URLError
   -> 다운로드 실패 (Timeout): http://recipe.ezmember.co.kr/cache/recipe/2018/07/19/b2b4a96aebe26e6d72a4350c35216eb01.jpg
   -> 다운로드 실패: http://www.megamart.com/attach/product_images/pro_ed_images/8801052/IMG_6591.jpg | 오류: HTTPError
   -> API 요청: 시작 위치 101
   -> 다운로드 실패: https://img1.daumcdn.net/thumb/R720x0.q80/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F9936CD4D5E6CFFFA44 | 오류: HTTPError
   -> 다운로드 실패: https://cdn.clien.net/web/api/file/F01/5803505/c031dcbe6ec51.jpg?w=780&h=30000&gif=true | 오류: URLError
   -> 다운로드 실패: https://thumb.ad.co.kr/ad/w/54/2b/a0/01/i/2079745.jpg | 오류: HTTPError
   -> 다운로드 실패 (Timeout): https://semie.cooking/image/board/cooking/hb/ce/bsgwbrjh/148167169gguz.jpg
   -> 다운로드 실패: http://down.humoruniv.com/hwiparambbs/data/pdswait/a_w5ae89b001_13edcf0dea